##### 第一模块(导入包模块：必须运行）

In [1]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import time
#time

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
# tk文件导入模块

##### 第二模块(文件处理模块：必须运行）

In [2]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")
        
def remote_select():
    print("以下文件可以调用，需要分析哪个文件？")
    print("-------------------------------------")
    index = 0
    list_item_temp = []
    for item in os.listdir():
        if ".xlsx" in item:
            index += 1
            print("[" + str(index) + "] " + item)
            list_item_temp.append(item)
    try:
        bash_pos = "/Users/dfuser/Desktop/目标文书目录/"
        file_code = int(input("需要导入哪个文件？(输入[]中的序号)"))
        final_pos = bash_pos + str(list_item_temp[file_code-1])
        table_ = pd.read_excel(final_pos)
        return table_
        print("表格导入成功，以下是表格预览")
        print("----------------------------")
    except Exception as e:
        print("导入错误")

def save_model_table1(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该摘要表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该分类表的名称(表1:主题 - 关键词表)(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass
    
def save_model_table2(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该摘要表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该分类表的名称(表2：各个文本对应的主题表)(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass

In [ ]:
##### 第三模块(LDA分类主逻辑)

In [3]:
def clean(single_para,stopwords_clean):
    """
    将stopwords和每一段文本取差集，清洗数据
    param single_para:没清洗文本集中的每一段数据
    return list_clean:每一段清洗好的文本
    """
    list_clean = set(single_para).difference(set(stopwords_clean))
    return list_clean

def data_prepare(corpus):
    """
    把数据清理干净
    param text_list:没清洗过的文本列表
    return list_clean:清洗过的文本列表
    """
    def segments(x):
        try:
            return jieba.lcut(x)
        except Exception as e:
            return "None"
    
    table_segments_list = list(map(segments,corpus))
    stopwords = list(map(lambda x:x.strip("\n"),codecs.open('stopwords.txt', 'r', 'utf-8').readlines()))
    stopwords_clean = list(map(lambda x:x.strip("\r"),stopwords))
    list_clean = list(map(clean,table_segments_list,stopwords_clean))
    return list_clean

def tfidf(clean_text):
    """ 
    产生tf-idf的模型
    param clean_text:清洗过的文本列表
    return corpus_tfidf:tf-idf模型
    """
    # 建立词典
    dictionary = corpora.Dictionary(clean_text)

    # 存档词典
    dictionary.save('dict_v1.dict')

    # 建立词袋模型
    corpus = [dictionary.doc2bow(text) for text in clean_text]

    # 建立tf-idf模型
    tfidf = models.TfidfModel(corpus)

    # 将词袋模型，转换为tf-idf模型
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf

# https://github.com/baidu/Familia/wiki
def lda(lda_model,corpus_tfidf,table_original_texts,num_topics):
    """
    使用lda算法进行分类
    param corpus_tfidf:tf-idf模型
    param table_original_texts:原始文本的列表
    param num_topics:分类多少个主题
    return table_keyword_classfication:关键词和关键词的分类表
    return table_originaltext_classfication:原文文档和关键词的分类表
    """
    # 关键词抽取 - 以及关键词属于哪一类
    top_words_per_topic = []
    for t in range(lda_model.num_topics):
        top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 10)])
    table_keyword_classfication = pd.DataFrame(top_words_per_topic)
    #table_keyword_classfication.to_excel("keyword_classification_words.xlsx")
    
    # 原始文章的归类
    list_classification_f = []
    for x in [item for item in lda_model.get_document_topics(corpus_tfidf)]:
        list_possibility = list(map(lambda x:x[1],x))
        list_classification = list(map(lambda x:x[0],x))
        list_possibility_index = list_possibility.index(max(list_possibility))
        result = list_classification[list_possibility_index]
        list_classification_f.append(result)
    table_original_text_classfication = pd.DataFrame([list(table_original_texts),list_classification_f]).T
    return table_keyword_classfication,table_original_text_classfication

def extract_something(pattern,elem):
    try:
        extract_infos = re.search(pattern,elem).group()
        return extract_infos
    except Exception as e:
        return "None"
    
def run_lda_model(table,selected_column):
    """
    运行上述框架
    param table:导入的表格
    return keyword_classfication:为每类预测主题下的关键词
    return original_text_classfication:为每个文本所可能归为的类别
    """
    corpus = table[selected_column]
    if selected_column == "判决结果":
        pattern_judgements = re.compile("判决如下:(.*?)。")
        corpus_judgement = list(map(extract_something,len(corpus) * [pattern_judgements],corpus))
        clean_text = data_prepare(corpus_judgement)
        corpus_tfidf = tfidf(clean_text)
        dictionary = corpora.Dictionary(clean_text)
        num_topics = input("请输入需要分类的主题数量:")
        lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics, iterations=500)
        keyword_classfication,original_text_classfication = lda(lda_model,corpus_tfidf,corpus_judgement,num_topics) #产生这两个表
    else:
        clean_text = data_prepare(corpus)
        corpus_tfidf = tfidf(clean_text)
        dictionary = corpora.Dictionary(clean_text)
        num_topics = input("请输入需要分类的主题数量:")
        lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics, iterations=500)
    
        keyword_classfication,original_text_classfication = lda(lda_model,corpus_tfidf,corpus,num_topics) #产生这两个表
        ### original_text_classfication：为每句话所归为的类别
        ### keyword_classfication：为每类主题下的关键词
    
    keyword_classfication.columns = ["属于分类","分类关键词","Weight"]
    original_text_classfication.columns = ["文本","属于分类"]
    
    return keyword_classfication,original_text_classfication

#### 控制部分

In [6]:
if __name__ == '__main__':
    # ------ 导入函数开始
    flag_input = int(input("你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)"))
    if flag_input == 1:
        root = Tk() # 实例化TKinter窗口
        root.withdraw() # 隐藏TKinter窗口
        table_ready_to_eat = import_table(root)
    elif flag_input == 2:
        table_ready_to_eat = remote_select()
    else:
        logging.error("加载错误")
    # ------ 导入函数结束
    
    print("表格中所有的列:",table_ready_to_eat.columns)
    selected_column = input("请问需要分析哪一列的关键词词频？(输入关键词部分):")
    
    keyword_classfication,original_text_classfication = run_lda_model(table_ready_to_eat,selected_column)
    save_model_table1(keyword_classfication)
    save_model_table2(original_text_classfication)

你是在本机上操作？还是远程操作？(1:本机操作,2:远程操作)2
以下文件可以调用，需要分析哪个文件？
-------------------------------------
[1] 夫妻债务 - 原表.xlsx
[2] wordfreqs.xlsx
[3] ex1.xlsx
[4] 校园暴力 - 结构化重构.xlsx
[5] 校园暴力事件 - 原表.xlsx
[6] 正当防卫 - 原表.xlsx
[7] 夫妻债务.xlsx
[8] .~zhaiwu_regex_filter.xlsx.xlsx
[9] 职业打假人 - 原表.xlsx
需要导入哪个文件？(输入[]中的序号)1
表格中所有的列: Index(['标题', '案号', '案件类型', '庭审程序', '案由', '文书类型', '法院', '判决日期', '原告', '被告',
       '第三人', '法官', '审判长', '审判员', '书记员', '头部', '头部2', '当事人', '当事人2', '庭审程序说明',
       '庭审程序说明2', '庭审过程', '庭审过程2', '庭审过程3', '庭审过程4', '庭审过程5', '庭审过程6', '法院意见',
       '法院意见2', '判决结果', '判决结果2', '庭后告知', '庭后告知2', '结尾', '结尾2', '附录', '附录2'],
      dtype='object')
请问需要分析哪一列的关键词词频？(输入关键词部分):法院意见


W1115 16:48:39.186950 4552865216 smart_open_lib.py:385] this function is deprecated, use smart_open.open instead


请输入需要分类的主题数量:5


W1115 16:48:50.882311 4552865216 ldamodel.py:934] too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


-------------------------------------------
以下为表格的预览:
   属于分类 分类关键词    Weight
0     0     n  0.004909
1     0     o  0.004903
2     0     N  0.004898
3     0     e  0.004867
4     0    家庭  0.000530
是否需要保存该摘要表？(输入Y/N):N
-------------------------------------------
以下为表格的预览:
                                                  文本 属于分类
0  本院认为，本案的争议焦点为:1.蔡秀英向张兰英提供100000元借款，是否属于官建坤与张兰英...    3
1  本院再审认为，根据《最高人民法院关于审理涉及夫妻债务纠纷案件适用法律有关问题的解释》第三条规...    3
2  本院再审认为，胡宇新再审期间向本院提交的雷若寒银行账户明细，具有中国农业银行股份有限公司天津...    0
3                                                NaN    0
4  本院认为:本案的焦点一是债务是否真实，二是是否属于夫妻共同债务。、关于债务的真实性，根据原审...    0
是否需要保存该摘要表？(输入Y/N):N


In [8]:
original_text_classfication

,文本,属于分类
0,本院认为，本案的争议焦点为:1.蔡秀英向张兰英提供100000元借款，是否属于官建坤与张兰英...,3
1,本院再审认为，根据《最高人民法院关于审理涉及夫妻债务纠纷案件适用法律有关问题的解释》第三条规...,3
2,本院再审认为，胡宇新再审期间向本院提交的雷若寒银行账户明细，具有中国农业银行股份有限公司天津...,0
3,NaN,0
4,本院认为:本案的焦点一是债务是否真实，二是是否属于夫妻共同债务。、关于债务的真实性，根据原审...,0
5,NaN,0
6,本院再审认为，债务应当清偿。、刘影向徐忠国借款95万元，有刘影出具的借据为证，双方借贷关系真...,3
7,本院认为，於艳娜对35份借条的真实性存有异议，且该35份借条均为复印件，本院对其真实性不予确...,3
8,本院再审认为，本案争议焦点是:1.王佳佳应否对王龙登记为经营者期间的银座国际会所借款承担偿还...,0
9,本院再审认为，申诉人任徽与原审被告赵清保虽然在赵清保举债期间是夫妻关系，但赵清保所借款项是用...,0
